In [2]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

In [3]:
small_noise_url = "art_daily_small_noise.csv"
df_small_noise = pd.read_csv(small_noise_url, parse_dates=True, index_col="timestamp")
print(df_small_noise.shape)

daily_jumpsup_url = "art_daily_jumpsup.csv"
df_daily_jumpsup = pd.read_csv(daily_jumpsup_url, parse_dates=True, index_col="timestamp")
print(df_daily_jumpsup.shape)

(4032, 1)
(4032, 1)


## Quick look at the data

In [4]:
print(df_small_noise.head())
print(df_daily_jumpsup.head())

                         value
timestamp                     
2014-04-01 00:00:00  18.324919
2014-04-01 00:05:00  21.970327
2014-04-01 00:10:00  18.624806
2014-04-01 00:15:00  21.953684
2014-04-01 00:20:00  21.909120
                         value
timestamp                     
2014-04-01 00:00:00  19.761252
2014-04-01 00:05:00  20.500833
2014-04-01 00:10:00  19.961641
2014-04-01 00:15:00  21.490266
2014-04-01 00:20:00  20.187739


In [5]:
fig, ax = plt.subplots()
df_small_noise.plot(legend=False, ax=ax)
plt.show()

In [6]:
fig, ax = plt.subplots()
df_daily_jumpsup.plot(legend=False, ax=ax)
plt.show()

## Prepare training data

A `value` for every 5 mins for 14 days.

-   24 * 60 / 5 = **288 timesteps per day**
-   288 * 14 = **4032 data points** in total

In [7]:
# Normalize
training_mean = df_small_noise.mean()
training_std = df_small_noise.std()
df_training_value = (df_small_noise - training_mean) / training_std
print("Number of training samples:", len(df_training_value))

Number of training samples: 4032


### Create sequences

In [8]:
TIME_STEPS = 288

# Generated training sequences for use in the model.
def create_sequences(values, time_steps=TIME_STEPS):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)


x_train = create_sequences(df_training_value.values)
print("Training input shape: ", x_train.shape)

Training input shape:  (3745, 288, 1)


## Build a model


In [9]:
model = keras.Sequential([layers.Input(shape=(x_train.shape[1], x_train.shape[2])),
        layers.Conv1D(filters=32, kernel_size=7, padding="same", strides=2, activation="relu"),
        layers.Conv1D(filters=16, kernel_size=7, padding="same", strides=2, activation="relu"),
        layers.Conv1DTranspose(filters=16, kernel_size=7, padding="same", strides=2, activation="relu"),
        layers.Conv1DTranspose(filters=32, kernel_size=7, padding="same", strides=2, activation="relu"),
        layers.Conv1DTranspose(filters=1, kernel_size=7, padding="same")])
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 144, 32)           256       
                                                                 
 conv1d_1 (Conv1D)           (None, 72, 16)            3600      
                                                                 
 conv1d_transpose (Conv1DTra  (None, 144, 16)          1808      
 nspose)                                                         
                                                                 
 conv1d_transpose_1 (Conv1DT  (None, 288, 32)          3616      
 ranspose)                                                       
                                                                 
 conv1d_transpose_2 (Conv1DT  (None, 288, 1)           225       
 ranspose)                                                       
                                                        

## Train the model


In [10]:
history = model.fit(x_train,x_train,
    epochs=50,
    batch_size=128,
    validation_split=0.1,
    callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")])

Epoch 1/50
27/27 [==============================] - 2s 40ms/step - loss: 0.6602 - val_loss: 0.0763
Epoch 2/50
27/27 [==============================] - 1s 30ms/step - loss: 0.0675 - val_loss: 0.0431
Epoch 3/50
27/27 [==============================] - 1s 33ms/step - loss: 0.0376 - val_loss: 0.0306
Epoch 4/50
27/27 [==============================] - 1s 34ms/step - loss: 0.0296 - val_loss: 0.0246
Epoch 5/50
27/27 [==============================] - 1s 31ms/step - loss: 0.0244 - val_loss: 0.0206
Epoch 6/50
27/27 [==============================] - 1s 32ms/step - loss: 0.0210 - val_loss: 0.0182
Epoch 7/50
27/27 [==============================] - 1s 32ms/step - loss: 0.0188 - val_loss: 0.0164
Epoch 8/50
27/27 [==============================] - 1s 32ms/step - loss: 0.0170 - val_loss: 0.0148
Epoch 9/50
27/27 [==============================] - 1s 35ms/step - loss: 0.0151 - val_loss: 0.0130
Epoch 10/50
27/27 [==============================] - 1s 33ms/step - loss: 0.0133 - val_loss: 0.0114
Epoch 11/

In [11]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()

## Detecting anomalies


In [12]:
# Get train MAE loss.
x_train_pred = model.predict(x_train)
train_mae_loss = np.mean(np.abs(x_train_pred - x_train), axis=1)

plt.hist(train_mae_loss, bins=50)
plt.xlabel("Train MAE loss")
plt.ylabel("No of samples")
plt.show()

# Get reconstruction loss threshold.
threshold = np.max(train_mae_loss)
print("Reconstruction error threshold: ", threshold)

118/118 [==============================] - 1s 6ms/step
Reconstruction error threshold:  0.023653634768429847


### Compare recontruction.

In [13]:
# Checking how the first sequence is learnt
plt.plot(x_train[0])
plt.plot(x_train_pred[0])
plt.show()

In [14]:
# Checking how the first sequence is learnt
plt.plot(x_train[100])
plt.plot(x_train_pred[100])
plt.show()

### Prepare test data

In [15]:
df_test_value = (df_daily_jumpsup - training_mean) / training_std
fig, ax = plt.subplots()
df_test_value.plot(legend=False, ax=ax)
plt.show()

# Create sequences from test values.
x_test = create_sequences(df_test_value.values)
print("Test input shape: ", x_test.shape)

Test input shape:  (3745, 288, 1)


In [16]:
# Get test MAE loss.
x_test_pred = model.predict(x_test)
test_mae_loss = np.mean(np.abs(x_test_pred - x_test), axis=1)
test_mae_loss = test_mae_loss.reshape((-1))

plt.hist(test_mae_loss, bins=50)
plt.xlabel("test MAE loss")
plt.ylabel("No of samples")
plt.show()

118/118 [==============================] - 1s 6ms/step


In [17]:
# Detect all the samples which are anomalies.
anomalies = test_mae_loss > threshold
print("Number of anomaly samples: ", np.sum(anomalies))
print("Indices of anomaly samples: ", np.where(anomalies))

Number of anomaly samples:  395
Indices of anomaly samples:  (array([2701, 2702, 2703, 2704, 2705, 2706, 2707, 2708, 2709, 2710, 2711,
       2712, 2713, 2714, 2715, 2716, 2717, 2718, 2719, 2720, 2721, 2722,
       2723, 2724, 2725, 2726, 2727, 2728, 2729, 2730, 2731, 2732, 2733,
       2734, 2735, 2736, 2737, 2738, 2739, 2740, 2741, 2742, 2743, 2744,
       2745, 2746, 2747, 2748, 2749, 2750, 2751, 2752, 2753, 2754, 2755,
       2756, 2757, 2758, 2759, 2760, 2761, 2762, 2763, 2764, 2765, 2766,
       2767, 2768, 2769, 2770, 2771, 2772, 2773, 2774, 2775, 2776, 2777,
       2778, 2779, 2780, 2781, 2782, 2783, 2784, 2785, 2786, 2787, 2788,
       2789, 2790, 2791, 2792, 2793, 2794, 2795, 2796, 2797, 2798, 2799,
       2800, 2801, 2802, 2803, 2804, 2805, 2806, 2807, 2808, 2809, 2810,
       2811, 2812, 2813, 2814, 2815, 2816, 2817, 2818, 2819, 2820, 2821,
       2822, 2823, 2824, 2825, 2826, 2827, 2828, 2829, 2830, 2831, 2832,
       2833, 2834, 2835, 2836, 2837, 2838, 2839, 2840, 2841, 2

In [18]:
# data i is an anomaly if samples [(i - timesteps + 1) to (i)] are anomalies
anomalous_data_indices = []
for data_idx in range(TIME_STEPS - 1, len(df_test_value) - TIME_STEPS + 1):
    if np.all(anomalies[data_idx - TIME_STEPS + 1 : data_idx+1]):
        anomalous_data_indices.append(data_idx)

In [19]:
df_subset = df_daily_jumpsup.iloc[anomalous_data_indices]
fig, ax = plt.subplots()
df_daily_jumpsup.plot(legend=False, ax=ax)
df_subset.plot(legend=False, ax=ax, color="r")
plt.show()

In [99]:
i = anomalous_data_indices[0]
print(i, test_mae_loss[i])

2988 0.08203310428591688


In [72]:
th = 50
data = np.array([22, 23, 31, 56, 66, 54, 25, 22, 31, 19])
anomaly_data = data > th

In [73]:
print(anomaly_data)
print(anomaly_data.shape)

[False False False  True  True  True False False False False]
(10,)


In [74]:
TIME_STEPS = 3
LENGTH = 10
p = TIME_STEPS - 1

for data_idx in range(p, LENGTH - p):
    print(data_idx)

2
3
4
5
6
7


In [75]:
for data_idx in range(p, LENGTH - p):
    print(data_idx, data[data_idx - p : data_idx+1])

2 [22 23 31]
3 [23 31 56]
4 [31 56 66]
5 [56 66 54]
6 [66 54 25]
7 [54 25 22]


In [76]:
[22, 23, 31, 56, 66, 54, 25, 22, 31, 19]

[22, 23, 31, 56, 66, 54, 25, 22, 31, 19]

In [78]:
indices = []
for data_idx in range(p, LENGTH - p):    
    print(anomaly_data[data_idx - p : data_idx + 1])
    if np.all(anomaly_data[data_idx - p : data_idx + 1]):        
        indices.append(data_idx)
        
print(indices)

[False False False]
[False False  True]
[False  True  True]
[ True  True  True]
[ True  True False]
[ True False False]
[5]


In [ ]:
np.all([False,  True,  True])